In [13]:
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [14]:
transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))             
    ])

In [15]:
train_data = torchvision.datasets.CIFAR10(root='./data',train=True, transform=transform,download=True)
test_data = torchvision.datasets.CIFAR10(root='./data',train=False, transform=transform,download=True)

Files already downloaded and verified
Files already downloaded and verified


In [16]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

In [17]:
image, label = train_data[0]

In [18]:
image.size()

torch.Size([3, 32, 32])

In [19]:
class_names = ['plane', "car", "bird", 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [20]:
class NeuralNet(nn.Module) :

    def __init__(self) :
        super().__init__()

        self.conv1 = nn.Conv2d(3, 12, 5) # (12, 28, 28)
        self.pool = nn.MaxPool2d(2,2) # (12 ,14 ,14)
        self.conv2 = nn.Conv2d(12, 24, 5) # (24, 10 ,10) -> (24, 5, 5) -> flatten (24 *5 *5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x) :
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        


In [21]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [24]:
for epoch in range(30) :
    print("Training epoch : " + str(epoch))

    running_loss = 0.0

    for i, data in enumerate(train_loader) :
        inputs, labels = data
        optimizer.zero_grad()

        outputs = net(inputs)

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print("Loss : " + str(running_loss/len(train_loader)))

Training epoch : 0
Loss : 2.153355517527726
Training epoch : 1
Loss : 1.721635686001256
Training epoch : 2
Loss : 1.5159487661000481
Training epoch : 3
Loss : 1.4023679266811866
Training epoch : 4
Loss : 1.3069697007946837
Training epoch : 5
Loss : 1.222743075700883
Training epoch : 6
Loss : 1.1536976610401541
Training epoch : 7
Loss : 1.0950955692881281
Training epoch : 8
Loss : 1.039083084385897
Training epoch : 9
Loss : 0.9893641875710002
Training epoch : 10
Loss : 0.9425715951300248
Training epoch : 11
Loss : 0.901571829427303
Training epoch : 12
Loss : 0.8664670223352319
Training epoch : 13
Loss : 0.830161060553976
Training epoch : 14
Loss : 0.7964697992161002
Training epoch : 15
Loss : 0.7664374751649602
Training epoch : 16
Loss : 0.7417914032058996
Training epoch : 17
Loss : 0.7153473004117198
Training epoch : 18
Loss : 0.6822608586446033
Training epoch : 19
Loss : 0.661184339952713
Training epoch : 20
Loss : 0.6393570795760121
Training epoch : 21
Loss : 0.6139125955158217
Train

In [25]:
torch.save(net.state_dict(), 'trained_net.pth')

In [ ]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

In [28]:
correct = 0
total = 0

net.eval()

with torch.no_grad() :
    for data in test_loader :
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print("Accuracy = " + str(accuracy))
     

Accuracy = 68.5


In [35]:
# TEST WITH ACTUAL IMAGES

new_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def load_image(image_path) :
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image


image_paths = ['Horse.jpg','Frog.jpg','Chat.jpg','Car.jpg']
images = [load_image(img) for img in image_paths]

net.eval()
with torch.no_grad() :
    for image in images :
        output = net(image)
        _, predicted = torch.max(output, 1)
        print(f'Prediction : {class_names[predicted.item()]}')

Prediction : horse
Prediction : frog
Prediction : cat
Prediction : car
